In [3]:
import requests
from functools import lru_cache
import time
from random import choice

Декоратор `@clock` для измерения времени выполнения (взял из семинара)

In [2]:
def clock(func):
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start

        print(f'Function executed in {total_time:.3f}s')
        return res
        
    return clocked

Для удобства создал декоратор `@history`, который в активном режиме будет хранить историю последних `size` вызовов функции

In [4]:
def history(active=True, size=128):
    def decorate(func: callable):
        history = [None for _ in range(size)] if active else []
        def decorated_function(arg):
            res = func(arg)
            if active:
                history.pop(0)
                history.append(arg)

                print(f'Last {size} calls:')
                for i in range(size - 1, -1, -1):
                    print(f'{size - i}) {"-" if (history[i] is None) else history[i]}')
            return res
        return decorated_function
    return decorate

Теперь создадим основной декоратор `cache_decorator` \\
`cache_type = 1` - кэширование по методу Least Recently Used (LRU) \\
`cache_type = 2` - кэширование N последних вызовов


In [6]:
def cache_decorator(cache_type, N=None, K=None):
    def decorate(func: callable):
        if N is None:
            return func

        last_calls = dict()
        max_size = None if N == -1 else N
        @history(active=K is not None, size=K)
        @lru_cache(maxsize=max_size if cache_type == 1 else 0)
        def decorated_function(arg):
            if cache_type == 1:
                return func(arg)

            r = last_calls[arg] if (arg in last_calls) else func(arg)
            if len(last_calls) == N and arg not in last_calls:
                k = next(iter(last_calls))
                last_calls.pop(k)
            last_calls[arg] = r
            return r

        return decorated_function
    return decorate

Протестируем работу созданного декоратора. Создадим список email-ов и будем тестировать работу функции (вызова API с сайта МИЭМ). Данная функция выдает полное ФИО преподавателя по его корпоративной почте. Если же почты не существует, выводит "Not found"

In [7]:
emails = ['vbashun@miem.hse.ru', 'vminchenkov@hse.ru', 'uq3dtr3rxuq3@hse.ru', 'oevsyutin@hse.ru', 'kzhanashia@hse.ru', 'sartamonov@miem.hse.ru']

Сама "heavy calculations" функция (пока без декораторов):

In [ ]:
def get_miem_teacher_full_name(email):
    time.sleep(1)
    name, domain = email.split('@')
    r = requests.get(
        f'https://devcabinet.miem.vmnet.top/public-api/user/email/{name}%40{domain}')
    if r.json()['data'] is None:
        return 'Not found'
    else:
        return r.json()['data']['fullName']

Проверять будем для 10 рандомных e-mailов с различными методами кеширования \\
1) Тестируем для LRU с размером 3 и историей последних 2 вызовов:

In [8]:
@clock
@cache_decorator(cache_type=1, N=3, K=2)
def get_miem_teacher_full_name(email):
    time.sleep(1)
    name, domain = email.split('@')
    r = requests.get(
        f'https://devcabinet.miem.vmnet.top/public-api/user/email/{name}%40{domain}')
    if r.json()['data'] is None:
        return 'Not found'
    else:
        return r.json()['data']['fullName']

for i in range(10):
    print(get_miem_teacher_full_name(choice(emails)))
    print()

Last 2 calls:
1) vbashun@miem.hse.ru
2) -
Function executed in 1.900s
Башун Владимир Владимирович

Last 2 calls:
1) sartamonov@miem.hse.ru
2) vbashun@miem.hse.ru
Function executed in 1.688s
Артамонов Сергей Юрьевич

Last 2 calls:
1) sartamonov@miem.hse.ru
2) sartamonov@miem.hse.ru
Function executed in 0.000s
Артамонов Сергей Юрьевич

Last 2 calls:
1) uq3dtr3rxuq3@hse.ru
2) sartamonov@miem.hse.ru
Function executed in 1.672s
Not found

Last 2 calls:
1) uq3dtr3rxuq3@hse.ru
2) uq3dtr3rxuq3@hse.ru
Function executed in 0.001s
Not found

Last 2 calls:
1) vbashun@miem.hse.ru
2) uq3dtr3rxuq3@hse.ru
Function executed in 0.000s
Башун Владимир Владимирович

Last 2 calls:
1) oevsyutin@hse.ru
2) vbashun@miem.hse.ru
Function executed in 1.677s
Евсютин Олег Олегович

Last 2 calls:
1) vminchenkov@hse.ru
2) oevsyutin@hse.ru
Function executed in 1.688s
Минченков Виктор Олегович

Last 2 calls:
1) oevsyutin@hse.ru
2) vminchenkov@hse.ru
Function executed in 0.001s
Евсютин Олег Олегович

Last 2 calls:
1) oev

2) Теперь протестируем, чтобы во 1 не отображалась история при определенном К, а также отсутствовало какое либо кеширование при определенном N (любые запросы выполняются долго вне зависимости от истории прежних вызовов)

In [9]:
@clock
@cache_decorator(cache_type=1)
def get_miem_teacher_full_name(email):
    time.sleep(1)
    name, domain = email.split('@')
    r = requests.get(
        f'https://devcabinet.miem.vmnet.top/public-api/user/email/{name}%40{domain}')
    if r.json()['data'] is None:
        return 'Not found'
    else:
        return r.json()['data']['fullName']

for i in range(10):
    print(get_miem_teacher_full_name(choice(emails)))
    print()

Function executed in 1.862s
Башун Владимир Владимирович

Function executed in 1.675s
Евсютин Олег Олегович

Function executed in 1.714s
Башун Владимир Владимирович

Function executed in 1.714s
Артамонов Сергей Юрьевич

Function executed in 1.675s
Not found

Function executed in 1.673s
Башун Владимир Владимирович

Function executed in 1.684s
Минченков Виктор Олегович

Function executed in 1.650s
Башун Владимир Владимирович

Function executed in 1.693s
Евсютин Олег Олегович

Function executed in 1.666s
Евсютин Олег Олегович



Наконец, протестируем кастомное кэширование (только последних N элементов)

In [10]:
@clock
@cache_decorator(cache_type=2, N=3, K=3)
def get_miem_teacher_full_name(email):
    time.sleep(1)
    name, domain = email.split('@')
    r = requests.get(
        f'https://devcabinet.miem.vmnet.top/public-api/user/email/{name}%40{domain}')
    if r.json()['data'] is None:
        return 'Not found'
    else:
        return r.json()['data']['fullName']

for i in range(10):
    print(get_miem_teacher_full_name(choice(emails)))
    print()

Last 3 calls:
1) sartamonov@miem.hse.ru
2) -
3) -
Function executed in 1.722s
Артамонов Сергей Юрьевич

Last 3 calls:
1) kzhanashia@hse.ru
2) sartamonov@miem.hse.ru
3) -
Function executed in 1.695s
Not found

Last 3 calls:
1) oevsyutin@hse.ru
2) kzhanashia@hse.ru
3) sartamonov@miem.hse.ru
Function executed in 1.679s
Евсютин Олег Олегович

Last 3 calls:
1) uq3dtr3rxuq3@hse.ru
2) oevsyutin@hse.ru
3) kzhanashia@hse.ru
Function executed in 1.676s
Not found

Last 3 calls:
1) sartamonov@miem.hse.ru
2) uq3dtr3rxuq3@hse.ru
3) oevsyutin@hse.ru
Function executed in 1.730s
Артамонов Сергей Юрьевич

Last 3 calls:
1) sartamonov@miem.hse.ru
2) sartamonov@miem.hse.ru
3) uq3dtr3rxuq3@hse.ru
Function executed in 0.000s
Артамонов Сергей Юрьевич

Last 3 calls:
1) oevsyutin@hse.ru
2) sartamonov@miem.hse.ru
3) sartamonov@miem.hse.ru
Function executed in 0.000s
Евсютин Олег Олегович

Last 3 calls:
1) sartamonov@miem.hse.ru
2) oevsyutin@hse.ru
3) sartamonov@miem.hse.ru
Function executed in 0.000s
Артамонов С

Как мы видим, все работает правильно!